In [ ]:
from tqdm import tqdm
import pickle
import torch
import einops
from typing import Callable, Optional
import math
import os

from circuits.utils import (
    collect_activations_batch,
    get_nested_folders,
    to_device,
)
import circuits.eval_sae_as_classifier as eval_sae
import circuits.chess_utils as chess_utils
import circuits.othello_utils as othello_utils
import circuits.test_board_reconstruction as test_board_reconstruction
import circuits.othello_engine_utils as othello_engine_utils

In [ ]:
from huggingface_hub import hf_hub_download

repo_dir = '/home/can/chess-gpt-circuits'
autoencoder_group_name = "/othello_layer5_ef4/"


# download data from huggingface if needed
if not os.path.exists(f'{repo_dir}/autoencoders{autoencoder_group_name}'):
    hf_hub_download(repo_id='adamkarvonen/chess_saes', filename=f'{autoencoder_group_name.strip("/")}.zip', local_dir=f'{repo_dir}/autoencoders')
    # unzip the data
    os.system(f'unzip {repo_dir}/autoencoders/{autoencoder_group_name.strip("/")}.zip -d autoencoders')


In [ ]:
autoencoder_group_path = f"{repo_dir}/autoencoders/othello_layer5_ef4/"
autoencoder_folder = "ef=4_lr=1e-03_l1=6e-02_layer=5/"
autoencoder_path = autoencoder_group_path + autoencoder_folder
feature_labels_file = "indexing_None_n_inputs_10000_results_feature_labels.pkl"
reconstruction_file = "indexing_None_n_inputs_10000_results_reconstruction.pkl"
device = "cuda"
# device = "cpu" # Not sure wtf is going on, I get this error with CPU:
# RuntimeError: Unhandled FakeTensor Device Propagation for aten.bmm.default, found two different devices cpu:0, cpu
device = torch.device(device)
othello = eval_sae.check_if_autoencoder_is_othello(autoencoder_group_path)

n_inputs = 100
batch_size = 1

print(f"Othello: {othello}")
model_name = eval_sae.get_model_name(othello)

torch.set_printoptions(precision=2, sci_mode=False)
torch.set_grad_enabled(False)

# torch.set_default_tensor_type('torch.FloatTensor')  # sets default tensor type to CPU

In [ ]:
with open(autoencoder_path + feature_labels_file, "rb") as f:
    feature_labels = pickle.load(f)
feature_labels = to_device(feature_labels, device)

custom_functions = [othello_utils.games_batch_to_state_stack_mine_yours_BLRRC]

data = eval_sae.construct_dataset(
    othello, custom_functions, n_inputs,
    split="train", device=device)

data, ae_bundle, pgn_strings, encoded_inputs = eval_sae.prep_firing_rate_data(
        autoencoder_path, batch_size, "", model_name, data, device, n_inputs, othello
    )
ae_bundle.buffer = None

In [ ]:
thresholds_TF11 = feature_labels["thresholds"].to(device)
alive_features_F = feature_labels["alive_features"].to(device)
num_features = len(alive_features_F)
T, F, _, _ = thresholds_TF11.shape
indexing_function = None

if feature_labels["indexing_function"] in chess_utils.supported_indexing_functions:
    indexing_function = chess_utils.supported_indexing_functions[
        feature_labels["indexing_function"]
    ]

print(f"Num alive features: {num_features}")
print(f"Indexing function: {indexing_function}")

In [ ]:
start = 0
end = 3
feature_batch_size = 1
num_feature_iters = math.ceil(num_features / feature_batch_size)
game_of_interest = 0
move_of_interest = 30

pgn_strings_BL = pgn_strings[start:end]
encoded_inputs_BL = encoded_inputs[start:end]
encoded_inputs_BL = torch.tensor(encoded_inputs_BL).to(device)

results = test_board_reconstruction.initialize_reconstruction_dict(
        custom_functions, thresholds_TF11.shape[0], alive_features_F, device
    )

batch_data = eval_sae.get_data_batch(
    data, pgn_strings_BL, start, end, custom_functions, device
)

all_activations_FBL, encoded_token_inputs = collect_activations_batch(
    ae_bundle, encoded_inputs_BL, alive_features_F
)

if indexing_function is not None:
    all_activations_FBL, batch_data = eval_sae.apply_indexing_function(
        pgn_strings[start:end], all_activations_FBL, batch_data, device, indexing_function
    )

constructed_boards = test_board_reconstruction.initialized_constructed_boards_dict(
    custom_functions, batch_data, thresholds_TF11, device
)

feature_piece_counts_TF = torch.zeros(T, F, device=device)

# For thousands of features, this would be many GB of memory. So, we minibatch.
for feature in range(num_feature_iters):
    f_start = feature * feature_batch_size
    f_end = min((feature + 1) * feature_batch_size, num_features)
    f_batch_size = f_end - f_start

    activations_FBL = all_activations_FBL[
        f_start:f_end
    ]  # NOTE: Now F == feature_batch_size

    results, additive_boards = test_board_reconstruction.aggregate_feature_labels(
        results,
        feature_labels,
        custom_functions,
        activations_FBL,
        thresholds_TF11[:, f_start:f_end, :, :],
        f_start,
        f_end,
        device,
    )

    additive_board_TBLRRC = additive_boards[custom_functions[0].__name__]

    counts_per_threshold_T = einops.reduce(additive_board_TBLRRC[:, game_of_interest, move_of_interest, :, :, :], "T R1 R2 C -> T", "sum")

    feature_piece_counts_TF[:, f_start] = counts_per_threshold_T


    for custom_function in constructed_boards:
        constructed_boards[custom_function] += additive_boards[custom_function]
results = test_board_reconstruction.compare_constructed_to_true_boards(
    results, custom_functions, constructed_boards, batch_data, device
)
results = test_board_reconstruction.calculate_F1_scores(results, custom_functions)

In [ ]:
f1_scores = results[custom_functions[0].__name__]["f1_score"]
print(f1_scores)
best_idx = f1_scores.argmax()
print(f"Best threshold: {best_idx}")

In [ ]:
k = 30
top_20_features = torch.argsort(feature_piece_counts_TF[best_idx], descending=True)[:k]
print(f"Top 20 features: {top_20_features}")
print(f"Top 20 feature counts: {feature_piece_counts_TF[best_idx, top_20_features]}")

print(feature_piece_counts_TF[best_idx, top_20_features[0]].sum().item())

print(f"Top 20 feature labels shape: {feature_labels[custom_functions[0].__name__][best_idx, top_20_features].shape}")

top_feature_RRC = feature_labels[custom_functions[0].__name__][best_idx, top_20_features[0]]

print(top_feature_RRC.sum().item())

In [ ]:
non_classified_idx = -2

top_features_FRRC = feature_labels[custom_functions[0].__name__][best_idx, top_20_features[:k]]

output_RR = torch.ones((8,8)).to(device) * non_classified_idx

for i in range(k):
    topk_feature_RRC = top_features_FRRC[i]
    topk_feature_RR = torch.argmax(topk_feature_RRC, dim=-1)
    topk_feature_RR -= 1
    zero_positions = torch.all(topk_feature_RRC == 0, dim=-1)
    topk_feature_RR[zero_positions] = non_classified_idx

    for r in range(8):
        for c in range(8):
            if topk_feature_RR[r, c] == non_classified_idx:
                continue
            if output_RR[r, c] == non_classified_idx:
                output_RR[r, c] = i
            else:
                output_RR[r, c] = -3

print(output_RR)




In [ ]:
top_feature_RR = torch.argmax(top_feature_RRC, dim=-1)
top_feature_RR -= 1
zero_positions = torch.all(top_feature_RRC == 0, dim=-1)
top_feature_RR[zero_positions] = non_classified_idx

print("Top feature:")
for row in top_feature_RR:
    for value in row:
        # Print a blank space if the value is non_classified_idx, otherwise print the value
        if value.item() == non_classified_idx:
            print(' ', end=' ')
        else:
            print(value.item(), end=' ')
    print()  # Newline after each row

In [ ]:
board_state_RRC = batch_data[custom_functions[0].__name__][game_of_interest][move_of_interest]
board_state_RR = torch.argmax(board_state_RRC, dim=-1) - 1
print(board_state_RR)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_othello_board(board):
    """
    Plots an Othello board using matplotlib with a specific color lookup for different values.

    Args:
    board (torch.Tensor): A 2D tensor representing the Othello board,
                          where 0, -1, 1, and -2 are mapped to specific colors.
    """
    # Create a color map with specific colors
    # Creating a dictionary for the color mapping
    color_map = {-1: 'black', 0: 'grey', 1: 'white', non_classified_idx: 'yellow', -3: 'red'}
    
    # Replace board values with corresponding colors using a numpy vectorized operation
    label_colors = np.vectorize(color_map.get)(board.numpy())

    # Create a figure and axis for the plot
    fig, ax = plt.subplots()

    # Create a color map based on the unique labels in the board
    unique_labels = np.unique(board)
    colors = [color_map[label] for label in unique_labels]
    cmap = plt.matplotlib.colors.ListedColormap(colors)

    # Map board values to indices in the unique labels
    board_indices = np.vectorize(lambda x: np.where(unique_labels == x)[0][0])(board.numpy())

    # Plot the board using imshow
    cax = ax.imshow(board_indices, cmap=cmap)

    # Create a color bar with the correct labels
    cbar = fig.colorbar(cax, ticks=range(len(unique_labels)))
    cbar.ax.set_yticklabels([color_map[label] for label in unique_labels])

    # Set the axis to be off since we don't need it for a game board representation
    ax.axis('off')

    # Add a title to the plot
    plt.title('Othello Board. Grey = Empty, Yellow = Not present in one hot vector')

    # Show the plot
    plt.show()

plot_othello_board(board_state_RR.to('cpu'))
plot_othello_board(top_feature_RR.to('cpu'))

In [ ]:
top_feature_RR.to('cpu')

In [ ]:
def plot_comparison_othello_board(true_board_RR, recon_board_RR=None):
    """
    Plots a comparison of the true and reconstructed Othello boards using matplotlib.

    Args:
    true_board (torch.Tensor): A 2D tensor representing the true Othello board.
    recon_board (torch.Tensor): A 2D tensor representing the reconstructed Othello board.
    """

    if recon_board_RR is None:
        recon_board_RR = torch.ones_like(true_board_RR) * non_classified_idx

    # Create a color map based on the unique labels in the board
    color_map = {non_classified_idx: 'grey', -1: 'red', 0: 'white', 1: 'blue'}
    cmap = plt.matplotlib.colors.ListedColormap(color_map.values())

    board_indices = recon_board_RR.numpy() - min(color_map.keys())


    # Create a figure and axis for the plot
    fig, ax = plt.subplots()
    plt.imshow(board_indices, cmap=cmap)
    # add circles on each square with black borders
    for i in range(8):
        for j in range(8):
            circle = plt.Circle((j, i), 0.3, color=color_map[true_board_RR[i, j].item()], fill=True)
            circle_edges = plt.Circle((j, i), 0.3, color='black', fill=False)
            plt.gca().add_artist(circle)
            plt.gca().add_artist(circle_edges)
            if recon_board_RR[i, j].item() != non_classified_idx:
                plt.gca().add_patch(plt.Rectangle((j-0.5, i-0.5), 1, 1, fill=False, edgecolor='black', lw=2))
    plt.title('True Board and Reconstruction by Single Feature')
    # plt.savefig('othello_board_comparison.png', dpi=300)
    plt.show()

plot_comparison_othello_board(board_state_RR.to('cpu'), top_feature_RR.to('cpu'))

In [ ]:
def plot_feature_attribution_board(board: torch.Tensor, k: int):
    # Creating a color gradient for top k features
    gradient_colors = plt.cm.viridis(np.linspace(0, 1, k))
    
    # Define special colors for no significant features and conflicts
    special_colors = {non_classified_idx: 'grey', -3: 'red'}
    
    # Combine all colors into a list
    colors = [special_colors.get(str(int(label)), gradient_colors[int(label)]) if label >= 0 else special_colors[int(label)]
              for label in np.unique(board)]
    
    # Create a color map from the list of colors
    cmap = plt.matplotlib.colors.ListedColormap(colors)
    
    # Map the board values to indices in the unique labels
    unique_labels = np.unique(board)
    board_indices = np.vectorize(lambda x: np.where(unique_labels == x)[0][0])(board.numpy())

    # Plot the board
    fig, ax = plt.subplots()
    cax = ax.imshow(board_indices, cmap=cmap)
    cbar = fig.colorbar(cax, ticks=range(len(unique_labels)))
    cbar.ax.set_yticklabels([f'Feature {int(label)}' if label >= 0 else ('No feature' if label == non_classified_idx else 'Multiple features') for label in unique_labels])

    # Hide the axes
    # ax.axis('off')

    # Title
    plt.title(f'Feature Attribution Board, top {k}')
    # plt.savefig(f'feature_attribution_board_top{k}.png', dpi=300)
    # Display the plot
    plt.show()

plot_feature_attribution_board(output_RR.to('cpu'), k)